## ArcGIS Processing Scripts

In [2]:
import os
import math

import numpy as np
import pandas as pd

import arcpy
import arcgis

import utils

%load_ext autoreload
%autoreload 2

In [3]:
config = utils.load_config("config.yml")

In [4]:
def features(op, island, source="nccos", year="2007", ext=None):
    if ext is None:
        return os.path.join(utils.ROOT(), source, year, "output", op, island)
    return "{}{}{}".format(os.path.join(utils.ROOT(), source, year, "output", op, island), os.extsep, ext)

### Trim polygons

In [ ]:
def polygon_trim(islands):
    for island in islands:
        path = utils.path_to_shoreline(island)
        pair = max(arcpy.da.TableToNumPyArray(path, ["OID@", "SHAPE@AREA"]), key=lambda p: p[1])
        with arcpy.da.UpdateCursor(path, ["OID@"]) as cursor:
            for row in cursor:
                if row[0] != pair[0]:
                    cursor.deleteRow()

### Buffer analysis – Polygon

https://desktop.arcgis.com/en/arcmap/10.3/tools/analysis-toolbox/buffer.htm

In [ ]:
def buffer_polygon(islands, config):
    size = config["pix_res"] * config["data_extraction"]["pix_dim"]

    for island in islands:
        
        in_features = utils.path_to_shoreline(island)
        out_features = features("polygon_buffer", island)
        
        os.makedirs(os.path.dirname(out_features), exist_ok=True)

        arcpy.Buffer_analysis(
            in_features, 
            out_features, 
            buffer_distance_or_field="{} METERS".format(size // 2), 
            line_side="OUTSIDE_ONLY", 
            dissolve_option="ALL"
        )
        
buffer_polygon(["oahu"], config)

### Eliminate polygon parts

https://desktop.arcgis.com/en/arcmap/10.3/tools/data-management-toolbox/eliminate-polygon-part.htm

In [ ]:
def eliminate_polygon_parts(islands, config):
    for island in islands:
        
        in_features = features("polygon_buffer", island, ext="shp")
        out_features = features("polygon_eliminate", island)

        os.makedirs(os.path.dirname(out_features), exist_ok=True)

        arcpy.EliminatePolygonPart_management(
            in_features, 
            out_features, 
            condition="AREA_OR_PERCENT",
            part_area_percent=99, 
            part_option="ANY"
        )

eliminate_polygon_parts(["oahu"], config)

### Generate points along line

https://pro.arcgis.com/en/pro-app/tool-reference/data-management/generate-points-along-lines.htm

In [ ]:
def generate_points_along_lines(islands, config):
    size = config["pix_res"] * config["data_extraction"]["pix_dim"]
    step = int(size * (1 - config["data_extraction"]["overlap"]))

    for island in islands:

        in_features = features("polygon_eliminate", island, ext="shp")
        out_features = features("point", island)

        os.makedirs(os.path.dirname(out_features), exist_ok=True)        
        
        arcpy.GeneratePointsAlongLines_management(
            in_features, 
            out_features,
            Point_Placement="DISTANCE", 
            Distance="{} METERS".format(step), 
        )

generate_points_along_lines(["oahu"], config)

### Buffer analysis – Point

https://desktop.arcgis.com/en/arcmap/10.3/tools/analysis-toolbox/buffer.htm

In [ ]:
def buffer_point(islands, config):
    size = config["pix_res"] * config["data_extraction"]["pix_dim"]

    for island in islands:
        
        in_features = features("point", island, ext="shp")
        out_features = features("point_buffer", island)
        
        os.makedirs(os.path.dirname(out_features), exist_ok=True)

        arcpy.Buffer_analysis(
            in_features, 
            out_features,
            buffer_distance_or_field="{} METERS".format(size // 2)
        )
        
buffer_point(["oahu"], config)

### Feature envelope to polygon

https://desktop.arcgis.com/en/arcmap/10.3/tools/data-management-toolbox/feature-envelope-to-polygon.htm


In [5]:
def feature_envelope_to_polygon(islands, config):
    for island in islands:
        in_features = features("point_buffer", island, ext="shp")
        out_features = features("rectangles", island)
        os.makedirs(os.path.dirname(out_features), exist_ok=True)
        arcpy.FeatureEnvelopeToPolygon_management(
            in_features,
            out_features
        )
        
envelope_to_polygon(["oahu"], config)

In [ ]:
rectangles = []

size = config["pix_res"] * config["data_extraction"]["pix_dim"] + config["data_extraction"]["pix_pad"]
step = int(size * (1 - config["data_extraction"]["overlap"]))

for island, polygon in polygons.items():
    
    rectangles.append([])
    
    polyline = polygon.boundary()
    polyline_perimeter = int(polyline.getLength("PLANAR", "METERS"))
    print("Perimeter: {0:.2f}".format(polyline_perimeter))

    spatial_reference = polygon.spatialReference
    point_A = arcpy.PointGeometry(polyline.firstPoint).projectAs(spatial_reference)
    
    for perimeter_position in range(0, polyline_perimeter, step):
        
        if len(rectangles[-1]) % 100 == 0:
            N, percentage = len(rectangles[-1]), perimeter_position / polyline_perimeter
            print("{0} rectangles collected, {1:.2f}% complete.".format(N, percentage))
        
        multipoint = point_A.buffer(size).boundary().intersect(polyline, 1)
                            
        point_B, _ = sorted(
            map(lambda pt: (pt, polyline.measureOnLine(pt)), multipoint), 
            key=lambda x: (x[1] < perimeter_position, x[1] - perimeter_position)
        )[0]

        point_B = arcpy.PointGeometry(point_B).projectAs(spatial_reference)
        
        assert math.isclose(point_A.distanceTo(point_B), size, rel_tol=10) # high rel_tol for high variance estimate
        
        line_AB = arcpy.Polyline(arcpy.Array([point_A.firstPoint, point_B.firstPoint])).projectAs(spatial_reference)
        
        point_C = utils.isoscelese_vertex_point(line_AB, size * 2.0)
        
        triangle = arcpy.Polyline(arcpy.Array([point_A.firstPoint, point_B.firstPoint, point_C.firstPoint]))
                
        rectangles[-1].append(triangle.hullRectangle)
        
        point_A = polyline.positionAlongLine(perimeter_position)
        